<a href="https://colab.research.google.com/github/feiduobaby/good-first-issue/blob/main/Capstone1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import polars as pl
import os
import joblib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp -r /content/drive/MyDrive/114239_nfl_competition_files_published_analytics_final /content/

In [4]:
# Paths and constants
input_path = '/content/114239_nfl_competition_files_published_analytics_final/train'
def read_selected_data(input_path):
    # Define the directory containing your data files

    # List three specific Parquet files you want to read
    selected_files = [f"input_2023_w0{i}.csv" for i in range(1,10)]+[f"input_2023_w{i}.csv" for i in range(10,19)]
    # Load and filter the data from only the selected Parquet files
    dfs = []
    for file_name in selected_files:
        file_path = f'{input_path}/{file_name}'
        lazy_df = pl.scan_csv(file_path)
        df = lazy_df.collect()
        dfs.append(df)

    # Concatenate all dataframes into a single dataframe
    full_df = pl.concat(dfs)

    return full_df

In [6]:
df=read_selected_data(input_path)
df.head()

game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,player_position,player_side,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
i64,i64,bool,i64,i64,str,i64,str,str,i64,str,str,str,str,f64,f64,f64,f64,f64,f64,i64,f64,f64
2023090700,101,false,54527,1,"""right""",42,"""Bryan Cook""","""6-1""",210,"""1999-09-07""","""FS""","""Defense""","""Defensive Coverage""",52.33,36.94,0.09,0.39,322.4,238.24,21,63.259998,-0.22
2023090700,101,false,54527,2,"""right""",42,"""Bryan Cook""","""6-1""",210,"""1999-09-07""","""FS""","""Defense""","""Defensive Coverage""",52.33,36.94,0.04,0.61,200.89,236.05,21,63.259998,-0.22
2023090700,101,false,54527,3,"""right""",42,"""Bryan Cook""","""6-1""",210,"""1999-09-07""","""FS""","""Defense""","""Defensive Coverage""",52.33,36.93,0.12,0.73,147.55,240.6,21,63.259998,-0.22
2023090700,101,false,54527,4,"""right""",42,"""Bryan Cook""","""6-1""",210,"""1999-09-07""","""FS""","""Defense""","""Defensive Coverage""",52.35,36.92,0.23,0.81,131.4,244.25,21,63.259998,-0.22
2023090700,101,false,54527,5,"""right""",42,"""Bryan Cook""","""6-1""",210,"""1999-09-07""","""FS""","""Defense""","""Defensive Coverage""",52.37,36.9,0.35,0.82,123.26,244.25,21,63.259998,-0.22


In [7]:
df.select(pl.all().is_null().any())

game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,player_position,player_side,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false


In [8]:
df.describe()

statistic,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,player_birth_date,player_position,player_side,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
str,f64,f64,f64,f64,f64,str,f64,str,str,f64,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",4.880579e6,4.880579e6,4.880579e6,4.880579e6,4.880579e6,"""4880579""",4.880579e6,"""4880579""","""4880579""",4.880579e6,"""4880579""","""4880579""","""4880579""","""4880579""",4.880579e6,4.880579e6,4.880579e6,4.880579e6,4.880579e6,4.880579e6,4.880579e6,4.880579e6,4.880579e6
"""null_count""",0.0,0.0,0.0,0.0,0.0,"""0""",0.0,"""0""","""0""",0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",2.0232e9,2196.408924,0.267067,49558.903219,16.131788,null,60.550452,null,null,211.278286,null,null,null,null,60.500736,26.811895,3.019878,2.118335,180.497185,181.536618,11.64147,60.51581,26.637664
"""std""",201140.46685,1246.426122,null,5210.337609,11.130079,null,23.059354,null,null,22.177471,null,null,null,null,23.489191,10.006199,2.227939,1.415794,100.716153,98.009122,5.331537,25.296428,15.438138
"""min""",2.0231e9,54.0,0.0,30842.0,1.0,"""left""",11.0,"""A'Shawn Robinson""","""5-10""",153.0,"""1984-05-19""","""CB""","""Defense""","""Defensive Coverage""",0.41,0.62,0.0,0.0,0.0,0.0,5.0,-5.26,-3.91
"""25%""",2.0231e9,1150.0,null,45198.0,8.0,null,41.0,null,null,195.0,null,null,null,null,42.63,18.99,1.09,1.01,90.91,91.74,8.0,42.610001,13.3
"""50%""",2.0231e9,2171.0,null,52413.0,15.0,null,61.0,null,null,207.0,null,null,null,null,60.41,26.85,2.72,1.92,179.56,180.14,10.0,60.509998,26.469999
"""75%""",2.0231e9,3246.0,null,54500.0,22.0,null,80.0,null,null,225.0,null,null,null,null,78.23,34.62,4.62,3.04,270.83,271.58,14.0,78.470001,39.869999
"""max""",2.0240e9,5258.0,1.0,56673.0,123.0,"""right""",109.0,"""Zyon McCollum""","""6-9""",358.0,"""2002-10-05""","""WR""","""Offense""","""Targeted Receiver""",119.86,52.88,12.53,17.12,360.0,360.0,94.0,125.849998,57.330002


In [9]:
df.select(
    pl.col("player_to_predict").value_counts(sort=True)
).unnest("player_to_predict")

player_to_predict,count
bool,u32
false,3577139
true,1303440


In [10]:
df.group_by(
    (pl.col("play_id")).alias("play_id"),
    maintain_order=True,
).len()

play_id,len
i64,u32
101,2993
194,1614
219,1333
361,1320
436,1095
…,…
2852,948
920,234
3679,348
